# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on Structured Streaming (Kafka integration)** </center>
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Spark Session creation


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://f42d033e597b:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ab30315c-9fe7-46bd-882d-8055d8f24d00;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Kafka Stream creation

In [3]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "1a3c6adb4591:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [4]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

### Sink configuration

In [5]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(120)

25/04/09 01:34:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bfdb9750-78c7-4a00-8532-71f30c5d5849. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/09 01:34:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/09 01:34:57 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+---------+
|key|value|topic|partition|offset|timestamp|timestampType|value_str|
+---+-----+-----+---------+------+---------+-------------+---------+
+---+-----+-----+---------+------+---------+-------------+---------+



25/04/09 01:34:59 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3382 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+----+-------------------+-------------------+---------+------+-----------------------+-------------+---------+
|key |value              |topic              |partition|offset|timestamp              |timestampType|value_str|
+----+-------------------+-------------------+---------+------+-----------------------+-------------+---------+
|NULL|[68 6F 6C 61]      |kafka-spark-example|0        |0     |2025-04-09 01:35:02.078|0            |hola     |
|NULL|[70 72 75 65 62 61]|kafka-spark-example|0        |1     |2025-04-09 01:35:04.505|0            |prueba   |
+----+-------------------+-------------------+---------+------+-----------------------+-------------+---------+



25/04/09 01:35:10 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 4443 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----+----------------------------+-------------------+---------+------+-----------------------+-------------+---------+
|key |value                       |topic              |partition|offset|timestamp              |timestampType|value_str|
+----+----------------------------+-------------------+---------+------+-----------------------+-------------+---------+
|NULL|[63 6F 72 72 69 65 6E 64 6F]|kafka-spark-example|0        |2     |2025-04-09 01:35:06.387|0            |corriendo|
|NULL|[6C 6F 6C]                  |kafka-spark-example|0        |3     |2025-04-09 01:35:07.482|0            |lol      |
+----+----------------------------+-------------------+---------+------+-----------------------+-------------+---------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [6]:
query.stop()

In [7]:
sc.stop()